<a href="https://colab.research.google.com/github/bubbysolway/BreastDCEDL/blob/main/BrestDCEDL_zenodo_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BreastDCEDL
## BreastDCEDL Data Explore and Visualize
## Demo on Zenodo data - no need to download




#### Author: Bubby Solway
#### Date : 2025-09-22
> BreastDCEDL/BrestDCEDL_demo_on_local_data_min_crop.ipynb


In [ ]:
from sklearn.metrics import classification_report,auc,roc_auc_score
from PIL import Image
import time
from pathlib import Path


import os
import numpy as np
import pandas as pd
from PIL import Image


import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('Do not show this message')

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from glob import glob
#from skimage import io
from sklearn.utils import shuffle

#from nipype.interfaces.ants import N4BiasFieldCorrection
import sys
import os
import ast
import re
import nibabel as nib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def show_n_images(
    imgs,
    cmap="gray",
    titles=None,
    enlarge=4,
    mtitle=None,
    cut=False,
    axis_off=True,
    fontsize=15,
    cb=False,
):
    """Visualise *n* images side-by-side."""
    _ = plt.figure(figsize=(enlarge * len(imgs), enlarge * 2))
    for i, im in enumerate(imgs, 1):
        ax = plt.subplot(1, len(imgs), i)
        if cb and len(np.unique(im)) > 5:
            im = cont_br(im)
        ax.imshow(im[50:290, 75:450] if cut else im, cmap=cmap, origin="lower")
        if titles is not None:
            ax.set_title(titles[i - 1], fontsize=fontsize)
        if axis_off:
            ax.axis("off")
    if mtitle:
        plt.suptitle(mtitle, fontsize=fontsize + 2)
    plt.tight_layout(); plt.show()


In [ ]:
# --------------------------------------------------------------------- #
#                       NIfTI I/O utilities                             #
# --------------------------------------------------------------------- #

def _ds_from_pid(pid: str) -> str:
    if "ISPY1" in pid:
        return "spy1"
    if ("ISPY2" in pid) or ("ACRIN-6698" in pid):
        return "spy2"
    if "Breast_MRI" in pid:
        return "duke"
    raise ValueError(f"Cannot infer dataset for pid={pid!r}")




def read_nifti(path: str) -> np.ndarray:
    return nib.load(path).get_fdata()

def _last_int(path):
    m = re.search(r"(\d+)\.nii\.gz$", path)
    return int(m.group(1)) if m else -1


def get_all_nifti_acquisitions(pid):



    dpath = nifti_path


    ff = [x for x in os.listdir(dpath) if pid in x]
    if len(ff)==0: return None
    if deb:print(os.listdir(dpath)[:7],pid)
    if deb:print(ff)


    img = []
    for f in ff:
        x = read_nifti(os.path.join(dpath, f))
        img.append(x)

    return img

def get_sorted_nifti_acquisitions(pid):



    ff = [x for x in os.listdir(nifti_path) if pid in x]
    if len(ff)==0: return None
    if deb:print(os.listdir(nifti_path)[:7],pid)
    if deb:print(ff)
    sorted_filenames = sorted(ff, key=_last_int)

    img = []
    for f in sorted_filenames:
        x = read_nifti(os.path.join(nifti_path, f))
        img.append(x)

    return img


def get_nifti_mask(pid: str):


    mpath=mask_path

    ff = [x for x in os.listdir(mpath) if pid in x]

    if deb:print(ff)
    if len(ff)==0: return None

    f=ff[0]


    x = read_nifti(os.path.join(mpath, f))


    return x


In [ ]:
def minmax(m):
    if m.max()==0:
        return m
    m = (m-m.min())/(m.max()-m.min())
    return m

def to_rgb(a,b,c):

    stack_axis = a.ndim

    # Stack along the next axis
    im = np.stack([minmax(a), minmax(b), minmax(c)], axis=stack_axis)

    return im

# get data from zenodo and unzip

In [ ]:
!wget https://zenodo.org/record/17580237/files/BreastDCEDL_demo_data.tar.gz?download=1 -O BreastDCEDL_demo_data.tar.gz

In [ ]:
!wget https://zenodo.org/record/17580237/files/BreastDCEDL_metadata_min_crop.csv?download=1 -O BreastDCEDL_metadata_min_crop.csv

In [ ]:
!tar -xvzf BreastDCEDL_demo_data.tar.gz

In [ ]:
# set to base of your local nifti data
data_path="BreastDCEDL_demo_data"

In [ ]:
nifti_path = os.path.join(data_path,'dce')
mask_path = os.path.join(data_path,'mask')

## Load and view metada

In [ ]:
df=pd.read_csv('BreastDCEDL_metadata_min_crop.csv')
df.info()

## I-SPY-2 example

In [ ]:
nifti_path

In [ ]:
pids=[]
deb=1
for i,row in df.iterrows():

    if get_all_nifti_acquisitions(row['pid']) is not None:
        print(row['pid'])
        pids.append(row['pid'])

pids

In [ ]:
df[df.pid.isin(pids)]

In [ ]:
duke_pids=[p for p in pids if 'Breast_MRI' in p]


In [ ]:
deb=0

In [ ]:


        pid=pids[0]

        row = df[df.pid==pid].iloc[0]
        smask=int(row['mask_start'])
        emask = int(row['mask_end'])
        pmask = (smask+emask)//2
        nz=int(row['n_z'])
        nxy=int(row['n_xy'])


        d=get_sorted_nifti_acquisitions(pid)
        m = get_nifti_mask(pid)

        print('dce',nz,nxy,d[0].shape, 'mask: ', smask,emask,  m.shape)

        show_n_images([d[0][k] for k in range(0,nz,5)])
        show_n_images([d[1][k] for k in range(0,nz,5)])
        show_n_images([d[2][k] for k in range(0,nz,5)])
        show_n_images([m[k] for k in range(0,nz,5)])
        show_n_images([to_rgb(d[0][k],d[1][k], d[2][k]) for k in range(0,nz,5)])
        show_n_images([to_rgb(d[0][k],d[1][k], m[k]) for k in range(0,nz,5)])
        show_n_images([to_rgb(d[0][k],d[1][k], d[2][k]) for k in [pmask-3,pmask,pmask+3]])
        show_n_images([to_rgb(d[0][k],d[1][k], m[k]) for k in [pmask-3,pmask,pmask+3]])
        show_n_images([ m[k] for k in [pmask-3,pmask,pmask+3]])




In [ ]:
    for pid in pids[1:]:
        if pid in duke_pids: continue
        print('===',pid)

        row = df[df.pid==pid].iloc[0]
        smask=int(row['mask_start'])
        emask = int(row['mask_end'])
        pmask = (smask+emask)//2
        nz=int(row['n_z'])
        nxy=int(row['n_xy'])
        print('mask: ', smask,emask)

        d=get_sorted_nifti_acquisitions(pid)
        m = get_nifti_mask(pid)

        im=to_rgb(d[0],d[1], d[2])
        show_n_images([to_rgb(d[0][k],d[1][k], d[2][k]) for k in [pmask-1,pmask,pmask+1]])
        show_n_images([to_rgb(d[0][k],d[1][k], m[k]) for k in [pmask-1,pmask,pmask+1]])
        show_n_images([ m[k] for k in [pmask-1,pmask,pmask+1]])





### DUKE example

In [ ]:

    for pid in pids[1:]:
        if pid not in duke_pids: continue
        print('===',pid)

        bbox=df[df.pid==pid][['mask_start', 'mask_end',
           'sraw', 'eraw', 'scol', 'ecol']].values[0].astype(int)
        print(bbox)
        d=get_sorted_nifti_acquisitions(pid)
        print(len(d),d[0].shape)
        m=np.zeros(d[0].shape)
        m[bbox[0]:bbox[1], bbox[2]:bbox[3], bbox[4]:bbox[5]]=255
        idx=[bbox[0]+1,(bbox[0]+bbox[1])//2,bbox[1]-1]
        show_n_images([d[0][k] for k in idx],axis_off=True)
        show_n_images([d[1][k] for k in idx],axis_off=True)
        show_n_images([d[2][k] for k in idx],axis_off=True)
        show_n_images([m[k] for k in idx],axis_off=True)

        show_n_images([to_rgb(d[0][k],
                                    d[1][k],d[2][k]) for k in idx],axis_off=True)
        show_n_images([to_rgb(d[0][k],
                                    d[1][k],m[k]) for k in idx],axis_off=True)

        show_n_images([to_rgb(d[0][k],
                                    d[1][k],d[2][k]) for k in idx])
